In [1]:
import sys
sys.path.append('../src/CTA-data-analisis-library/')

In [2]:
#cargamos librerias 
import os 
import subprocess
from datetime import datetime
import numpy as np 
import glob
import matplotlib.pyplot as plt
import tensorflow as tf 
import psutil
import re
import random
import shutil
import pickle

#propias
import unzipdata_and_first_treatments as manipulate
import loaddata4use
import model_creation_functions as models

In [3]:
#enviroment variables
npy_final_dir="../datos/elementos_npy"
base_dir_elementos="../datos/elementos"
elements=['gamma', 'electron', 'proton', 'helium', 'iron', 'nitrogen', 'silicon']

In [4]:
def runs_disponibles(npy_dir,elements):
    lista=[]
    for i in elements:
        npy_element_dir=os.path.join(npy_dir,"npy_"+i)
        runs=[int(re.search("run_([0-9]{3})_0\.npy",i).group(1)) for i in os.listdir(npy_element_dir) if re.search("run_([0-9]{3})_0\.npy",i)]
        lista.append(runs)
    return lista


def new_create_main_list_runs(number_runs_per_element,posibles_runs):
    #esto es aleatorio por defecto, porque es lo unico que necesito por ahora    
    final=[]
    for ind,lista_runs_element in enumerate(chose_runs):
        final.append(random.sample(lista_runs_element,number_runs_per_element[ind]))
    return final

#tenemos que hacer un ligero cambio porque se estan cargando con los ejes cambiados
def cambiar_ejes_lista(lista):
    for i,j in enumerate(lista):
        lista[i]=np.swapaxes(j,1,2)
    return lista

In [5]:
chose_runs=runs_disponibles(npy_final_dir,elements)
chose_runs[0].remove(2)
chose_runs[0].remove(2)
chose_runs[0].remove(3)
chose_runs[0].remove(3)
chose_runs[0].remove(3)

In [6]:
opciones_filtros=[
    [[12,16],[32,64],[64,12]],
    [[12,16,32],[64,128],[128,64,32]],
    [[12,16],[32,64],[64,12]],
    [[12,16],[32,64],[64,12]],
    [[12,16],[32,64],[64,12]]
]

In [11]:
registro=open("../modelos/1_data_control.txt","at")
n=7 #repes de boostrap
#primer bucle para arquitecturas
for i,arch in enumerate(opciones_filtros):
    print(f"{i}: {arch} \n")
    modelo=models.model_multi_tel(classes=3,filtros=arch,last_dense=[20,5])
    modelo.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["acc","AUC","mean_squared_error"])
    registro.write(f"Con arquitectura: {arch} : \n")

    #segundo_bucle para boostrap
    for k in range(n):
        print(f"\n Boostrap {k+1} de {n} \n")

        list_runs=new_create_main_list_runs([2,6,6,6,6,6,6],chose_runs)
        registro.write(f"Boostrap {k} de {n},runs: {list_runs} \n")
        x_train_list,x_test_list,y_train_list,y_test_list=loaddata4use.load_dataset_completo(npy_final_dir,labels_asign=[0,1,2,2,2,2,2],elements=elements,
                                                                                             main_list_runs=list_runs,pre_name_folders="npy_",telescopes=[1,2,3,4],
                                                                                             test_size=0.05,same_quant="same",verbose=True,fill=True,categorical=True)
        x_train_list=cambiar_ejes_lista(x_train_list)
        x_test_list=cambiar_ejes_lista(x_test_list)

        
        hist=modelo.fit(x=x_train_list,y=y_train_list,epochs=5, validation_data=(x_test_list,y_test_list),batch_size=64)
    del x_train_list,x_test_list,y_train_list,y_test_list
    registro.write("\n")
    modelo.save(f"../modelos/1_modelo_filtro_{i}_boostrap_{k+1}.h5")
    with open(f"../modelos/performances/1_history_modelo_filtro_{i}_boostrap_{k+1}.pickle","wb") as pick:
        pickle.dump(hist,pick)
        
registro.close()

0: [[12, 16], [32, 64], [64, 12]] 


 Boostrap 0 de 5 

Load of names and common events
_______________
0 ;  Element:  gamma  , Runs:  [5, 27, 14, 17]  Shape of common events (tels,common events):  (4, 1166)
1 ;  Element:  gamma  , Runs:  [5, 27, 14, 17]  Shape of common events (tels,common events):  (4, 1111)
2 ;  Element:  gamma  , Runs:  [5, 27, 14, 17]  Shape of common events (tels,common events):  (4, 1103)
3 ;  Element:  gamma  , Runs:  [5, 27, 14, 17]  Shape of common events (tels,common events):  (4, 1116)
0 ;  Element:  electron  , Runs:  [87, 145, 151, 140, 10, 22, 1, 139, 120]  Shape of common events (tels,common events):  (4, 233)
1 ;  Element:  electron  , Runs:  [87, 145, 151, 140, 10, 22, 1, 139, 120]  Shape of common events (tels,common events):  (4, 232)
2 ;  Element:  electron  , Runs:  [87, 145, 151, 140, 10, 22, 1, 139, 120]  Shape of common events (tels,common events):  (4, 258)
3 ;  Element:  electron  , Runs:  [87, 145, 151, 140, 10, 22, 1, 139, 120]  Shape of co

KeyboardInterrupt: 